<a href="https://colab.research.google.com/github/rushikeshnakhate/PortfoliOpt/blob/master/multiple_dates_expected_return_df_visualization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
# # Mount Google Drive
# from google.colab import drive
# drive.mount('/content/drive')

# data_dir = Path("/content/drive/MyDrive/data")
# !pip install tqdm
# !pip install PyPortfolioOpt
# !pip install tabulate
# !pip install dtale
# !pip install -q data-table
# from google.colab import data_table

In [12]:
import os
from pathlib import Path
import pandas as pd
from tabulate import tabulate
data_dir = Path(r"D:\PortfoliOpt\data")

In [13]:
def load_all_pickles_to_dfs(directory):
    """
    Reads all .pkl files in the specified directory and loads each file into a DataFrame.
    
    Parameters:
    - directory (str): The path to the directory containing .pkl files.

    Returns:
    - dict: A dictionary where keys are file names without extensions and values are DataFrames.
    """
    dataframes = {}
    for filename in os.listdir(directory):
        if filename.endswith(".pkl"):
            file_path = os.path.join(directory, filename)
            try:
                df = pd.read_pickle(file_path)
                dataframes[filename[:-4]] = df  # Using filename without .pkl extension as the key
            except Exception as e:
                print(f"Error reading {filename}: {e}")
    return dataframes

In [14]:
from datetime import datetime, timedelta

def generate_month_date_ranges(year, months=None):
    if months is None:
        months = range(1, 13)  # Default to all months if none specified

    month_ranges = []
    for month in months:
        start_date = datetime(year, month, 1).date()
        if month == 12:
            end_date = (datetime(year + 1, 1, 1) - timedelta(days=1)).date()
        else:
            end_date = (datetime(year, month + 1, 1) - timedelta(days=1)).date()
        month_ranges.append((start_date, end_date))
    return month_ranges


def create_current_month_directory(start_date):
    current_month = start_date.strftime("%Y%m")
    current_month_dir = data_dir / current_month
    return current_month_dir

In [15]:
# if __name__ == "__main__":
#     year = 2023
#     alternate_months = [1]
#     month_ranges = generate_month_date_ranges(year, months=alternate_months)
#     for start_date, end_date in month_ranges:
#         all_pkl_file_dir = create_current_month_directory(start_date=start_date)
#         all_dfs = load_all_pickles_to_dfs(all_pkl_file_dir)
#         for file_name, df in all_dfs.items():
#             print(f"DataFrame from {file_name}.pkl")
#             print(tabulate(df, headers='keys', tablefmt='pretty'))
#             

In [16]:
df = pd.read_pickle(r"D:\PortfoliOpt\data\202301\all_optimized_df.pkl")
valid_weights_df = df[~df['Weights'].str.contains('error', na=False)]
print(tabulate(valid_weights_df, headers='keys', tablefmt='pretty'))

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [17]:
expected_return_all_type_df = pd.read_pickle(r"D:\PortfoliOpt\data\202301\expected_return_all_type.pkl")
print(tabulate(expected_return_all_type_df, headers='keys', tablefmt='pretty'))

+---------------+-----------------------+--------------------------------+---------------------+--------------------------+----------------------+---------------------+-----------------------+------------------------+----------------------+
|    ticker     |      ARIMAReturn      | ArithmeticMeanHistoricalReturn |     CAPMReturn      | CAGRMeanHistoricalReturn | EMAHistoricalReturn  |  FamaFrenchReturn   |  GordonGrowthReturn   | LinearRegressionReturn |   RiskParityReturn   |
+---------------+-----------------------+--------------------------------+---------------------+--------------------------+----------------------+---------------------+-----------------------+------------------------+----------------------+
| ADANIPORTS.NS |  0.35714181900075515  |       0.3462136635480445       | 0.7673764240267555  |    0.2634318639682989    |  0.6849848717199556  | 0.3762136635480445  |        0.4037         |   3.495464885023708    |  0.3462136635480445  |
| APOLLOHOSP.NS |  -0.47369912487927

In [18]:
valid_weights_df_errors = df[df['Weights'].str.contains('error', na=False)]
print(tabulate(valid_weights_df_errors, headers='keys', tablefmt='pretty'))

+------+------------------------+-------------------+--------------+------------------------------------------------------------------------------------------------------------------------+--------------------------------+--------------------------+----------------------------------------------------------------------------------------------------+
|      | Expected Annual Return | Annual Volatility | Sharpe Ratio |                                                        Weights                                                         |      Expected Return Type      |        Risk Model        |                                             Optimizer                                              |
+------+------------------------+-------------------+--------------+------------------------------------------------------------------------------------------------------------------------+--------------------------------+--------------------------+-------------------------------------------------

In [19]:
df1 = pd.read_pickle(
    r"D:\PortfoliOpt\data\202301\post_processing_weight.pkl")
valid_post_process = df1[~df1['Weights'].str.contains('error', na=False)]
valid_post_process.to_excel(r"D:\PortfoliOpt\data\output.xlsx", index=False)

In [22]:
performance_df = pd.read_pickle(r"D:\PortfoliOpt\data\2023\performance_metrics.pkl")
# print(tabulate(performance_df, headers='keys', tablefmt='pretty'))
performance_df.to_excel(r"D:\PortfoliOpt\data\performance_df.xlsx", index=False)